In [1]:
include("npa.jl")
NPA.test()

solve_relaxation(obj_vec, mk, mkqs, rs_vecs, model) = (-0.7499999923584808, Float64[])
solve_relaxation_dual(obj_vec, mk, mkqs, rs_vecs, model) = (-0.7499999996318656, Float64[])


(-0.7499999996318656, Float64[])

In [2]:
using JuMP
using COSMO
using SparseArrays
using LinearAlgebra
using DataStructures
using IterativeSolvers
using LinearMaps
using FastGaussQuadrature
using QuantumOptics
using DelimitedFiles
using Mosek
using MosekTools
using QuantumOptics
using NLsolve
using DiffResults
using ForwardDiff
const FD = ForwardDiff
using Optim
using SpecialFunctions

# Parameters

  $\varepsilon_{EC}^{com}$: Probability to abort the EC step when using the honest device

  $\varepsilon_{EC}$: Probability that EC does not produce identical keys (when not aborting)

  $\varepsilon_{EA}^{com}$: Probability for the honest device to fail parameter estimation, related to tolerance $\delta$

Probability  $\varepsilon_s$: Smoothing parameter of the min-entropy in privacy amplification step. Note that for the smoothing parameter in the entropy accumulation we use $\frac{\varepsilon_s}{4}$
 
  $\varepsilon_{PA}$: Parameter for privacy amplification

The resulting security parameters are:

  Completeness: $\varepsilon_{QKD}^c = \varepsilon_{EC}^{com} + \varepsilon_{EA}^{com} + \varepsilon_{EC}$

  Soundness (=Correctness & Secrecy): $\varepsilon_{QKD}^s = \varepsilon_{EC} + \varepsilon_{PA} + 2\varepsilon_{s}$

In [3]:
struct SecParams
    # Error correction
    eps_EC_com::Float64
    eps_EC::Float64

    # Parameter estimation
    eps_EA_com::Float64

    # Privacy Amplification
    eps_s::Float64
    eps_PA::Float64
end

In [4]:
completeness(sec_params::SecParams) = sec_params.eps_EC_com + sec_params.eps_EA_com + sec_params.EC
soundness(sec_params::SecParams) = sec_params.eps_EC + sec_params.eps_PA + 2*sec_params.eps_s

soundness (generic function with 1 method)

## Helper functions to feed the SDP to COSMO

In [5]:
function convert_constraint(mats)
    m, n = size(mats[1])
    triplets = (Vector{Int64}(), Vector{Int64}(), Vector{Float64}())
    for i in 2:length(mats)
        for j in 1:length(mats[i].nzval)
            row = mats[i].rows[j]
            col = mats[i].cols[j]
            val = mats[i].nzval[j]
            
            push!(triplets[1], row + (col - 1)*m)
            push!(triplets[2], i - 1)
            push!(triplets[3], val)
        end
    end
    vec(mats[1]), sparse(triplets[1], triplets[2], triplets[3], m*n, length(mats) - 1) 
end


function solve_with_cosmo(obj_vec, P, ms)
    model = COSMO.Model()
    n_vars = length(obj_vec)

    constraints = Vector{COSMO.Constraint{Float64}}()
    for m in ms
        b, A = convert_constraint(m)
        constraint = COSMO.Constraint(A, b, COSMO.PsdCone)
        push!(constraints, constraint)
    end

    # assemble and solve the model
    # settings = COSMO.Settings{Float64}(verbose=true, decompose=false, kkt_solver=CGIndirectKKTSolver)
    settings = COSMO.Settings{Float64}(verbose=false, decompose=false, eps_prim_inf=1e-6, eps_dual_inf=1e-6, eps_abs=1e-6, eps_rel=1e-6)
    assemble!(model, P, obj_vec, constraints, settings=settings)
    result = COSMO.optimize!(model);

    result.obj_val, result.x
end

solve_with_cosmo (generic function with 1 method)

## The Brown-Fawzi Hierarchy

In [6]:
function get_subs(As, Bs, Zs, Zds)
    subs = Vector{Tuple{NPA.Polynome{Float64}, NPA.Polynome{Float64}}}()
    # Projective constraints
    for x in [As; Bs]
        push!(subs, (x*x, x))
    end

    # Commutation constraints for measurements   @ Main ./In[120]:52
    for b in Bs
        for a in As
            push!(subs, (b*a, a*b))
        end
    end

    # Commutation with Eve's operators
    for x in [As; Bs]
        for z in [Zs; Zds]
            push!(subs, (z*x, x*z))
        end
    end
    
    subs
end


function get_extra_monomials(As, Bs, Zs, Zds)
    monos = Vector{NPA.Polynome{Float64}}()

    # Add ABZ
    for a in As
        for b in Bs
            for z in [Zs; Zds]
                push!(monos, a*b*z)
            end
        end
    end
    
    # Add monos appearing in objective function
    for (z, zd) in zip(Zs, Zds)
        push!(monos, As[1]*zd*z)
    end
    
    monos
end


function op_constraints(ti, As, Bs, Zs, Zds)
    constraints = Vector{NPA.Polynome{Float64}}()
    return constraints
end

op_constraints (generic function with 1 method)

In [7]:
struct BFFParams
    As::Vector{NPA.Polynome{Float64}}
    Bs::Vector{NPA.Polynome{Float64}}
    Zs::Vector{NPA.Polynome{Float64}}
    Zds::Vector{NPA.Polynome{Float64}}
    params::NPA.RelaxationParameters
    mk::Vector{NPA.SparseMatrixCOO{Float64, Int64}}
end

In [8]:
function BFFParams()
    m = 8                            # Number of nodes in gaussian quadrature
    vars = NPA.Variables()
    A0, A1 = NPA.create_hermitian_var(vars), NPA.create_hermitian_var(vars)
    B0, B1 = NPA.create_hermitian_var(vars), NPA.create_hermitian_var(vars)

    Zs = Vector{NPA.Polynome{Float64}}()
    Zds = Vector{NPA.Polynome{Float64}}()
    for i in 1:2
        Z, Zd = NPA.create_non_hermitian_var(vars)
        push!(Zs, Z)
        push!(Zds, Zd)
    end
    As = [A0, A1]
    Bs = [B0, B1]

    params = NPA.RelaxationParameters(
        k = 2,
        vars = vars,
        substitutions = get_subs(As, Bs, Zs, Zds),
        extra_monomials = get_extra_monomials(As, Bs, Zs, Zds),
    )
    mk = NPA.moment_matrices(params);
    @show length(mk)

    BFFParams(As, Bs, Zs, Zds, params, mk)
end

bff_params = BFFParams();

length(mk) = 804


In [9]:
function prob(x, y, a, b, sys, eta=1.0)
    basis = SpinBasis(1//2)
    psi0 = spinup(basis)
    psi1 = spindown(basis)

    id, sx, sz = one(basis), sigmax(basis), sigmaz(basis)
    theta, a0, a1, b0, b1, b2 = sys    
    phi = cos(theta)*tensor(psi0, psi0) + sin(theta)*tensor(psi1, psi1)
    rho = tensor(phi, dagger(phi))

    # Raw measurement operators
    a00 = 0.5*(id + cos(a0)*sz + sin(a0)*sx)
    a10 = 0.5*(id + cos(a1)*sz + sin(a1)*sx)

    # Inefficient measurements (binned to 0)
    A00 = eta*a00 + (1 - eta)*id
    A10 = eta*a10 + (1 - eta)*id
    As = [[A00, id - A00], [A10, id - A10]]

    # Raw measurement operators
    b00 = 0.5*(id + cos(b0)*sz + sin(b0)*sx)
    b10 = 0.5*(id + cos(b1)*sz + sin(b1)*sx)
    b20 = 0.5*(id + cos(b2)*sz + sin(b2)*sx)

    # Inefficient measurements (binned to 0)
    B00 = eta*b00 + (1 - eta)*id
    B10 = eta*b10 + (1 - eta)*id
    B20 = eta*b20
    B21 = eta*(id - b20)
    Bs = [[B00, id - B00], [B10, id - B10], [B20, B21, id - B20 - B21]]

    meas_op = tensor(As[x + 1][a + 1], Bs[y + 1][b + 1])
    (rho*meas_op) |> tr |> real |> Float64
end

prob (generic function with 2 methods)

In [10]:
function quadrature(m::Integer)
    t, w = gaussradau(m)
    (1.0 .- reverse(t))/2.0, reverse(w)/2.0
end


function objective(ti, q, As, Zs, Zds)
    obj = NPA.Polynome{Float64}()
    F = [As[1], 1.0 - As[1]]
    for a in 1:length(F)
        M = (1 - q)*F[a] + q*F[3 - a]
        obj += M*(Zs[a] + Zds[a] + (1 - ti)*Zds[a]*Zs[a]) + ti*Zs[a]*Zds[a]
    end
    obj
end


function get_statistics(sys, eta)
    stats = Float64[]

    # p(00|xy)
    for x in 0:1, y in 0:1
        push!(stats, prob(x, y, 0, 0, sys, eta))
    end
    
    # p(0|x)
    for x in 0:1
        p = 0.0
        for b in 0:1
            p += prob(x, 0, 0, b, sys, eta)
        end
        push!(stats, p)
    end
    
    # p(0|y)
    for y in 0:1
        p = 0.0
        for a in 0:1
            p += prob(0, y, a, 0, sys, eta)
        end
        push!(stats, p)
    end
    
    stats
end


function score_constraints(probs, As, Bs)
    i = 1
    constraints = NPA.Polynome{Float64}[]
    
    # Constraints on p(00|xy)
    for x in 0:1
        for y in 0:1
            push!(constraints, As[x + 1]*Bs[y + 1] - probs[i])
            i += 1
        end
    end
    
    # Marginal constraints on p(0|x)
    for x in 0:1
        push!(constraints, As[x + 1] - probs[i])
        i += 1
    end
    
    # Marginal constraints on p(0|y)
    for y in 0:1
        push!(constraints, Bs[y + 1] - probs[i])
        i += 1
    end
    
    constraints
end


function compute_entropy(params, probs, q, m)
    ent = 0.0
    t, w = quadrature(m)    # Nodes, weights of quadrature

    constraints = score_constraints(probs, params.As, params.Bs)
    rs_vecs = NPA.moment_vectors(params.params, constraints)
    rs_vecs = [rs_vecs; -rs_vecs]

    grad = zeros(length(constraints))
    for k in 1:m
        ck = w[k]/(t[k]*log(2))
        obj = objective(t[k], q, params.As, params.Zs, params.Zds)
        obj_vec = NPA.objective_vector(params.params, obj)
    
        model = Model(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true))
        dual, grad_k = NPA.solve_relaxation_dual(obj_vec, params.mk, [], rs_vecs, model; verbose=false)
        ent += ck*(1.0 + dual)

        grad += ck*(grad_k[1:length(grad)] - grad_k[length(grad) + 1:end])
    end
    
    ent, grad
end


function cond_ent(joint, marg)
    hab, hb = 0.0, 0.0

    for prob in joint
        if 0.0 < prob < 1.0
            hab += -prob*log2(prob)
        end
    end

    for prob in marg
        if 0.0 < prob < 1.0
            hb += -prob*log2(prob)
        end
    end
    hab - hb
end


function HAgB(sys, eta, q)
    probs = zeros(2, 3)
    for a in 1:2
        for b in 1:3
            pa = prob(0, 2, a - 1, b - 1, sys, eta)
            pa_bar = prob(0, 2, 2 - a, b - 1, sys, eta)
            probs[a, b] = (1 - q)*pa + q*pa_bar
        end
    end
    marginal = sum(probs, dims=1)
    cond_ent(reduce(vcat, probs), marginal)
end


function compute_rate(params, sys, eta, q, m)
    stats = get_statistics(sys, eta)
    ent, grad = compute_entropy(params, stats, q, m)
    err = HAgB(sys, eta, q)
    ent - err
end

compute_rate (generic function with 1 method)

In [11]:
test_sys = [π/4, 0, π/2, π/4, -π/4, 0]
test_eta = 0.99
test_q = 0.0

compute_rate(bff_params, test_sys, test_eta, test_q, 8)

0.8170017539910278

## Entropy Accumulation

In [12]:
function fixed_point(x0, f)
    x = copy(x0)
    delta = 1.0
    it = 0
    max_it = 12
    while delta >= 1e-5 && it < max_it
        x_new = f(x)
        dx = x_new - x
        delta = norm(x_new - x)
        x = copy(x_new)
        # x += 0.2*dx
        it += 1
    end
    
    if it == max_it
        println("Reached iteration limit")
        flush(stdout)
    end

    x
end


function conversion_matrix(γ)
    M = zeros(8, 16)
    M[1,  1: 4] = [4, 0, 0, 0] # p(00|00)
    M[2,  5: 8] = [4, 0, 0, 0] # p(00|01)
    M[3,  9:12] = [4, 0, 0, 0] # p(00|10)
    M[4, 13:16] = [4, 0, 0, 0] # p(00|11)

    M[5,  1: 4] = [4, 4, 0, 0] # p(a=0|x=0)
    M[6, 13:16] = [4, 4, 0, 0] # p(a=0|x=1)
    M[7,  1: 4] = [4, 0, 4, 0] # p(b=0|y=0)
    M[8, 13:16] = [4, 0, 4, 0] # p(b=0|y=1)

    N = zeros(16, 17)
    N[1:16, 1:16] = Matrix(I, 16, 16)/γ

    P = zeros(14, 17)
    P[1, 1:16] = [1, 1, 0, 0,   -1, -1,  0,  0,    0, 0,  0, 0,     0,  0,  0,  0] # p(a=0|0) = p(00|00) + p(01|00) = p(00|01) + p(01|01)
    P[2, 1:16] = [0, 0, 1, 1,    0,  0, -1, -1,    0, 0,  0, 0,     0,  0,  0,  0] # p(a=1|0) = p(10|00) + p(11|00) = p(10|01) + p(11|01)
    P[3, 1:16] = [0, 0, 0, 0,    0,  0,  0,  0,    1, 1,  0, 0,    -1, -1,  0,  0] # p(a=0|1) = p(00|10) + p(01|10) = p(00|11) + p(01|11)
    P[4, 1:16] = [0, 0, 0, 0,    0,  0,  0,  0,    0, 0,  1, 1,     0,  0, -1, -1] # p(a=1|1) = p(10|10) + p(11|10) = p(10|11) + p(11|11)

    P[5, 1:16] = [1, 0, 1, 0,    0,  0,  0,  0,  -1,  0, -1, 0,     0,  0,  0,  0] # p(b=0|0) = p(00|00) + p(10|00) = p(00|10) + p(10|10)
    P[6, 1:16] = [0, 1, 0, 1,    0,  0,  0,  0,   0, -1,  0, -1,    0,  0,  0,  0] # p(b=1|0) = p(01|00) + p(11|00) = p(01|10) + p(11|10)
    P[7, 1:16] = [0, 0, 0, 0,    1,  0,  1,  0,   0,  0,  0,  0,   -1,  0, -1,  0] # p(b=0|1) = p(00|01) + p(10|01) = p(00|11) + p(10|11)
    P[8, 1:16] = [0, 0, 0, 0,    0,  1,  0,  1,   0,  0,  0,  0,    0, -1,  0, -1] # p(b=1|1) = p(01|01) + p(11|01) = p(01|11) + p(11|11)

    P[ 9,  1: 4] = [1, 1, 1, 1] # p(x=0,y=0) = 0.25
    P[10,  5: 8] = [1, 1, 1, 1] # p(x=0,y=1) = 0.25
    P[11,  9:12] = [1, 1, 1, 1] # p(x=1,y=0) = 0.25
    P[12, 13:16] = [1, 1, 1, 1] # p(x=1,y=1) = 0.25

    P[13, :] .= 1  # total probability is 1
    P[14, end] = 1 # Probability of last element is fixed

    v = nullspace(P)
    proj = v*transpose(v)

    return M*N*proj
end


function eat_numeric(stats, bff_params, q, k; p0=stats)
    min_ent = Ref(0.0)
    ent_ = Ref(0.0)
    grad_ = stats |> length |> zeros |> Ref
    function f(p)
        ent, grad = compute_entropy(bff_params, copy(p), q, 8)

        # Compute the value and gradient of the error function
        result = DiffResults.GradientResult(grad)
        result = FD.gradient!(result, k, grad)
        err = DiffResults.value(result)
        v = DiffResults.gradient(result)

        min_ent.x = ent + dot(grad, stats - p) - err
        ent_.x = ent
        grad_.x = grad

        @assert err >= 0

        return stats - v
    end

    try
        res = fixedpoint(f, p0; xtol=1e-5, iterations=15, beta=0.7, m=5)
        p_star = res.zero
        return min_ent.x, ent_.x, grad_.x, p_star
    catch error
        # Sometimes NLsolve's fixedpoint method can run into issues so we provide our own backup
        println("Trying backup due to error: $error")
        flush(stdout)
        p_star = fixed_point(p0, f)
        return min_ent.x, ent_.x, grad_.x, p_star
    end
end

eat_numeric (generic function with 1 method)

In [13]:
etas = Iterators.flatten([LinRange(0.8, 0.85, 20)[1:end-1], LinRange(0.85, 0.95, 20)[1:end-1], LinRange(0.95, 1.0, 20)])
etas = collect(etas);

## Key rate computation

In [14]:
# Parameters used for evaluating quantum variance using COSMO
struct EATParams
    As::Vector{NPA.Polynome{Float64}}
    Bs::Vector{NPA.Polynome{Float64}}
    params::NPA.RelaxationParameters
    mk::Vector{NPA.SparseMatrixCOO{Float64, Int64}}
end

In [15]:
function get_subs(As, Bs)
    subs = Vector{Tuple{NPA.Polynome{Float64}, NPA.Polynome{Float64}}}()
    # Projective constraints
    for x in [As; Bs]
        push!(subs, (x*x, x))
    end

    # Commutation constraints for measurements   @ Main ./In[120]:52
    for b in Bs
        for a in As
            push!(subs, (b*a, a*b))
        end
    end

    return subs
end


function EATParams()
    m = 8                            # Number of nodes in gaussian quadrature
    vars = NPA.Variables()
    A0, A1 = NPA.create_hermitian_var(vars), NPA.create_hermitian_var(vars)
    B0, B1 = NPA.create_hermitian_var(vars), NPA.create_hermitian_var(vars)

    As = [A0, A1]
    Bs = [B0, B1]

    params = NPA.RelaxationParameters(
        k = 2,
        vars = vars,
        substitutions = get_subs(As, Bs),
        extra_monomials = [],
    )
    @time mk = NPA.moment_matrices(params);
    @show length(mk)

    return EATParams(As, Bs, params, mk)
end

eat_params = EATParams();

  0.000196 seconds (1.39 k allocations: 121.500 KiB)
length(mk) = 31


In [16]:
Phi(z) = (1 + erf(z/√2))/2
G(x, p) = max(x*log(x/p) + (1-x)*log((1-x)/(1-p)), 0)
function C(n, p, f)
    if f == 0
        return 0.0
    end
    if f == 1
        return 1.0
    end
    return Phi(sign(f - p)*sqrt(2*n*G(f, p)))
end


# Bisection method to find zeros
function bisect(f::Function, lo::Number, hi::Number; tol=1e-6)
    flo = f(lo)
    fhi = f(hi)
    
    if flo*fhi > 0
        error("No root in the interval ($lo, $hi)")
    end

    if flo > zero(typeof(lo)) && fhi < zero(typeof(hi))
        lo, hi = hi, lo
    end

    while abs(hi - lo) > tol
        mid = (lo + hi)/2
        fmid = f(mid)
        if fmid < zero(typeof(lo))
            lo = mid
        else
            hi = mid
        end
    end
    return (lo + hi)/2
end


# Compute a delta which produces a desired eps
function delta(n, p, eps, sign=1)
    if sign == 1
        x = bisect(f -> C(n, p, f) - (1 - eps), p, 1.0; tol=1e-11)
        delta = x - p
        return delta + 2/n
    else
        x = bisect(f -> C(n, p, f) - eps, 0.0, p; tol=1e-11)
        delta = p - x
        return delta + 2/n
    end
end

delta (generic function with 2 methods)

In [17]:
function obj_min(v, As, Bs)
    i = 1
    obj = NPA.Polynome{Float64}()
    Fs = [[As[1], 1 - As[1]], [As[2], 1 - As[2]]]
    Gs = [[Bs[1], 1 - Bs[1]], [Bs[2], 1 - Bs[2]]]

    for x in 0:1, y in 0:1
        for a in 0:1, b in 0:1
            obj += 0.25*v[i]*Fs[x+1][a+1]*Gs[y+1][b+1]
            i += 1
        end
    end

    return obj
end


# Computes the minimum over the quantum correlations p(a,b,x,y)
function min_quantum(params::EATParams, grad)
    obj_vec = NPA.objective_vector(params.params, obj_min(grad, params.As, params.Bs))

    # Pad the objective vec
    n_vars = max(length(obj_vec) - 1, length(params.mk) - 1)
    obj_padded = [obj_vec[2:end]; zeros(1 + n_vars - length(obj_vec))]
    
    P = spzeros(n_vars, n_vars)
    obj_val, minimizer = solve_with_cosmo(FD.value.(obj_padded), P, [params.mk])
    
    return obj_vec[1] + dot(minimizer, obj_padded)
end


function max_var_quantum(params::EATParams, v1, v2)
    obj_lin = NPA.objective_vector(params.params, obj_min(-v1, params.As, params.Bs))
    obj_quad = NPA.objective_vector(params.params, obj_min(v2, params.As, params.Bs))
    
    n_vars = max(length(obj_lin) - 1, length(obj_quad) - 1, length(params.mk) - 1)

    # Pad the objective vectors
    obj_lin_padded = [obj_lin[2:end]; zeros(1 + n_vars - length(obj_lin))]
    obj_quad_padded = [obj_quad[2:end]; zeros(1 + n_vars - length(obj_quad))]
    P = 2*obj_quad_padded*transpose(obj_quad_padded)
    
    # Add the linear term from the quadratic objective
    obj_lin_padded += 2*obj_quad[1]*obj_quad_padded
    
    obj_val, minimizer = solve_with_cosmo(FD.value.(obj_lin_padded), FD.value.(P), [params.mk])
    
    # Add the constant terms to the solution
    res = -(obj_lin[1] + obj_quad[1]^2 + dot(obj_lin_padded, minimizer) + dot(obj_quad_padded, minimizer)^2)
    return res
end

max_var_quantum(params::EATParams, grad) = max_var_quantum(params, grad.^2, grad)


function k_eat(eat_params::EATParams, grad, eps_s, eps_EA, n, γ, β, κ=1.0)
    dO = 2*3

    max_ = max(β, grad...)
    min_ = (1-γ)*β + min_quantum(eat_params, γ*grad)
    range = max_ - min_

    # var = (1-γ)*β^2 - ((1-γ)*β)^2 + max_var_quantum(eat_params, γ*grad.^2 - 2*(1-γ)*β*γ*grad, γ*grad)
    var = (1-γ)*γ*β^2 + max_var_quantum(eat_params, γ*grad.^2 - 2*(1-γ)*β*γ*grad, γ*grad)
    @assert var >= 0

    V = sqrt(var + 2) + log2(2*dO^2 + 1)
    α = 1 + κ*sqrt(2*log2(2/(eps_s^2*eps_EA^2)))/(sqrt(n*log(2))*V)
    @assert 1 < α < 2

    # We need to take care not to run into a numerical infinity
    K_α = (2-α)^3/(6*(3-2*α)^3*log(2)) * 2^((α-1)/(2-α)*(log2(dO)+range)) # The remaining terms will be (safely) added below
    if log2(dO) + range < 1_000.0
        K_α *= (log(2)*(log2(dO)+range) + log(1+ℯ^2/2^(log2(dO)+range)))^3
    else
        K_α *= (log(2)*(log2(dO)+range))^3
    end

    c1 = (α-1)/(2-α)*log(2)/2*V^2
    c2 = 1/(n*(α-1))*(log2(2/eps_s^2) + α*log2(1/eps_EA))
    c3 = ((α-1)/(2-α))^2*K_α

    return c1 + c2 + c3
end

# Extract the probability p(a,b,x,y) from the vector stats
function extract(stats, x, y, a, b)
    return stats[1 + b + 2*a + 4*y + 8*x]
end


# Compute H(A|B) for test rounds
function hab_test(stats)
    hab = 0.0
    for x in 0:1, y in 0:1
        probs = zeros(2, 2)
        for a in 0:1, b in 0:1
            probs[a+1, b+1] = 4*extract(stats, x, y, a, b)
        end
        marginal = sum(probs, dims=1)
        hab += 0.25*cond_ent(reduce(vcat, probs), marginal)
    end
    return hab
end


function EC_max(sec_params, γ, n, sys, η, q)
    eps_EC_com = sec_params.eps_EC_com
    eps_s_prime = 0.995*eps_EC_com

    stats = Float64[]
    for x in 0:1, y in 0:1
        for a in 0:1, b in 0:1
            push!(stats, 0.25*prob(x, y, a, b, sys, η))
        end
    end
    
    err = HAgB(sys, η, q)
    
    (1-γ)*err + γ*hab_test(stats) +
        2*log2(5)*sqrt(log2(2/eps_s_prime^2))/sqrt(n) +
        (2*log2(1/(eps_EC_com - eps_s_prime)) + 4)/n +
        log2(1/sec_params.eps_EC)/n
end


function error_term(sec_params::SecParams, γ, n, sys, η, q)
    eps_s = sec_params.eps_s
    eps_EC = sec_params.eps_EC
    eps_PA = sec_params.eps_PA
    
    dO = 2*2
    α = 1 + 1/sqrt(n)
    K_α = (2-α)^3/(6*(3-2*α)^3*log(2))*2^((α-1)/(2-α)*2*log2(dO))*log(2^(2*log(dO)) + ℯ^2)^3
    
    t1 = γ
    t2 = (log2(2/(eps_s/4)^2) + α*log2(1/(eps_PA + 2*eps_s)))/(n*(α-1))
    t3 = ((α-1)/(2-α))^2*K_α
    t4 = 2*log2(1/eps_PA)/n
    t5 = EC_max(sec_params, γ, n, sys, η, q)
    
    return t1 + t2 + t3 + t4 + t5
end


function EAT(sec_params::SecParams, γ, n, sys, η, q)
    stats = get_statistics(sys, η)

    probs = Float64[]
    for x in 0:1, y in 0:1
        for a in 0:1, b in 0:1
            push!(probs, γ*0.25*prob(x, y, a, b, sys, η))
        end
    end
    push!(probs, 1-γ)

    function k(grad)
        eps_s = sec_params.eps_s/4
        eps_EA = sec_params.eps_PA + 2*sec_params.eps_s
        β = -250.0

        S = conversion_matrix(γ)
        grad = transpose(S)*grad
        grad[end] = β

        abs_tol = Float64[]
        for i in 1:length(probs)
            p = probs[i]
            s = sign(grad[i])
            del = delta(n, p, sec_params.eps_EC_com/length(probs), s)
            @assert del >= 0
            push!(abs_tol, del)
        end

        function f(x)
            β, κ = x, 0.75

            grad = copy(grad)
            grad[end] = β

            res = k_eat(eat_params, grad[1:end-1], eps_s, eps_EA, n, γ, β, κ) + dot(abs.(grad), abs_tol)
            return res
        end

        # We choose to optimize the value of β. We could also fix β which is sometimes more stable
        res = Optim.optimize(f, -100_000.0, 100_000.0)
        return Optim.minimum(res)
    end
    return eat_numeric(stats, bff_params, q, k)
end


function rate(sec_params::SecParams, γ, n, η)
    x = readdlm("data/qkd_2322_8M_$(100_000*η |> floor |> Int).csv")
    q = x[3]
    sys = x[4:end]
    return rate(sec_params, γ, n, sys, η, q)
end


function rate(sec_params::SecParams, γ, n, sys, η, q)
    min_ent, ent, grad, p_star = EAT(sec_params, γ, n, sys, η, q)
    rate = min_ent - error_term(sec_params, γ, n, sys, η, q)
    return rate
end

rate (generic function with 2 methods)

In [18]:
γ = 1/300
eta = etas[19]
n = 1e11
@show eta

#                      eps_EC_com, eps_EC, eps_EA_com, eps_s, eps_PA
sec_params = SecParams(1e-2/2,     1e-10,  1e-2/2,     1e-10, 1e-10)

eta = 0.8473684210526315


SecParams(0.005, 1.0e-10, 0.005, 1.0e-10, 1.0e-10)

In [19]:
rate(sec_params, γ, n, eta)

0.0011633347570435548